<a href="https://colab.research.google.com/github/mlindsey19/evol-comp-two/blob/master/EvolComp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random
import numpy as np

N = 10 # must be even for crossover
RUNS = 30
LOCI = 3
LCBD = (-1.0, 5.0) #loci bounds
#print format -> #.###
float_formatter = lambda x: "%.5f" % x
np.set_printoptions( formatter = { 'float_kind' : float_formatter } )

gen = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( N, LOCI ) )


In [0]:
pen = 2
penFlag = 1

def fitness( individual ):
  fit = 0
  for x in individual:
    fit +=  pen * x**2 if ( penFlag and ( x > 5.0 or x < -1.0 ) ) else x**2
  return fit

def fitVector(g, n):
  fit = np.empty(n)
  for i in range(n):
    fit[ i ] = fitness( g[ i ] )
  return fit


def sortFit( g, n = N ):
  temp = np.c_[ g, fitVector( g, n ) ]
  return temp[ temp[ : , 3 ].argsort() ]

def getBestIndividual(g, n = N):
  return sortFit(g, n)[0]


In [0]:
Pc = 0.8
alpha = 0.7
shuffFlag = 1

def crossover(  ):
  l = np.random.randint(0,3) # 0-2 (3 is exclusive)
  if ( shuffFlag ):
    np.random.shuffle( gen )
  for i in range( N // 2 ):
    if ( np.random.uniform( 0, 1 ) < Pc ):
      j = 2 * i
      gen[ j ][ l ], gen[ j + 1 ][ l ] = gen[ j + 1 ][ l ],  gen[ j ][ l ]
#  return gen

def arithmeticCrossover(  ):
  if ( shuffFlag ):
    np.random.shuffle( gen )
  for i in range( N // 2 ):
    l = np.random.randint(0,3) # 0-2 (3 is exclusive)
    if ( np.random.uniform( 0, 1 ) < Pc ):
      j = 2 * i
      x = gen[ j + 1 ][ l ] + gen[ j ][ l ]
      gen[ j ][ l ], gen[ j + 1 ][ l ] = ( 1 - alpha ) * x, alpha * x
  #    return gen

def arithmeticCrossPerLoci(  ):
  if ( shuffFlag ):
    np.random.shuffle( gen )
  for i in range( N // 2 ):
    for h in range( 3 ):        
      j = 2 * i
      if ( np.random.uniform( 0, 1 ) < Pc ):
        x = gen[ j + 1 ][ h ] + gen[ j ][ h ]
        gen[ j ][ h ], gen[ j + 1 ][ h ] = ( 1 - alpha ) * x, alpha * x
 # return gen
    
  
 

***mr*** mutation range
***ub*** upper bound

The upper bound is calculated so that numbers selected at random fall in an appropriate 
range of mutation ***P*** % of the time 

In [0]:
Pm = 0.1
mr = 0.75

def mut():
  mtmt = np.empty( ( N, LOCI) )
  ub = ( 2 * mr / Pm ) - mr
  for i in range ( N ):
    for j in range(LOCI):
      m = np.random.uniform( (0 - mr), ub )
      if ( m < mr ):
        mtmt[i][j] = m
      else:
        mtmt[i][j] = 0  
  return mtmt

def gausIndi():
  mtmt = np.empty( ( N, LOCI) )

  for i in range ( N ):
    sd = np.std(gen[i])
    for j in range(LOCI):
      m = np.random.uniform( 0 , 1 )
      if ( m < Pm ):
        mtmt[i][j] = np.random.normal(0, sd)
      else:
        mtmt[i][j] = 0   
  return mtmt

def gausPop():
  sd = np.std(gen)
  mtmt = np.random.normal( 0.0, sd, ( N, LOCI) )

  for i in range ( N ):
    for j in range(LOCI):
      m = np.random.uniform( 0 , 1 )
      if ( m > Pm ):
            mtmt[i][j] = 0  
  return mtmt

def mutation(  ):
  global gen
  gen = gen + mut()

def gpmutation(  ):
  global gen
  gen = gen + gausPop()
def gimutation(  ):
  global gen
  gen = gen + gausIndi()


def mutationWithClamp(  ):
  global gen
  ngen = gen + mut()
  for rw in range( N ):
    for cl in range( LOCI ):  
        ngen[rw][cl] = ngen[rw][cl] if ( ngen[rw][cl] > -1 ) else -1
        ngen[rw][cl] = ngen[rw][cl] if ( ngen[rw][cl] < 5 ) else 5
  gen = ngen

  



In [0]:
Psbt = 0.8

def selectionDetBT(  ):
  global gen
  ngen = np.empty( ( N, LOCI) )
  for i in range(N):
    x = gen[ random.randint( 0, N - 1 ) ]
    y = gen[ random.randint( 0, N - 1 ) ]
    if( fitness( x ) > fitness( y ) ):
      x = y
    ngen[ i ] = x
  gen = ngen
  
def selectionProbBT(  ):
  global gen
  ngen = np.empty( ( N, LOCI) )
  p = np.random.uniform( 0, 1 )
  for i in range(N):
    x = gen[ random.randint( 0, N - 1 ) ]
    y = gen[ random.randint( 0, N - 1 ) ]
    if( fitness( x ) > fitness( y ) and p < Psbt ):
      x = y
    ngen[ i ] = x
  gen = ngen



In [0]:
min = .5
max = 2 - min
Plr = lambda r:( min + ( r - 1 ) / ( N - 1 ) * ( max - min ) ) / N
PlrMap = np.array( [ Plr(x) for x in range( 1, 11 ) ] ) 
pmap =np.empty(N)
for i in range(N):
  if (i == 0):
    pmap[ i ] = PlrMap[ i ]
  else:
    pmap[ i ] = PlrMap[ i ] + pmap[ i - 1 ]


def selectionLR(  ):
  global gen
  ngen = np.empty( (N, LOCI) )
  temp = sortFit( gen )
  for i in range( N ):
    s = np.random.uniform( 0, 1 )
    for idx,c in enumerate(pmap):
      if( s <= c ):
        #print(s, c , 9 - idx)
        ngen[ i ] = temp[ 9-idx, : 3 ] 
        break
  gen = ngen

en = 2  # number of best fit guarantee propogate  

def selectionEliteLR(  ):
  global gen
  ngen = np.empty( (N, LOCI) )
  temp = sortFit( gen )
  for i in range( en ):
    ngen[ i ] = temp[ i, : 3]
  for i in range( en, N ):
    s = np.random.uniform( 0, 1 )
    for idx,c in enumerate(pmap):
      if( s <= c ):
        #print(s, c , 9 - idx)
        ngen[ i ] = temp[ 9-idx, : 3 ] 
        break
  gen = ngen


In [0]:
tn = 8 # number to keep
firstRunFlag = 1
if ( firstRunFlag ):
  firstRunFlag = 0
  prevgen = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( N, LOCI ) )
 
def selectionTrunc(  ):
  global gen
  ngen = np.empty( ( N, LOCI ) )
  temp = sortFit( gen )
  for i in range( tn ):
    ngen[ i ] = temp[ i, : 3 ]
  for i in range ( tn, N ):
    ngen [ i ] = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( 1, LOCI ) )
  gen = ngen
 
def selectionFibTrunc(  ):
  global gen
  ngen = np.empty( ( N, LOCI ) )
  global prevgen
  temp = gen
  union = np.append(gen, prevgen, axis = 0)
  union = sortFit(union, n=(2 * N) )
  prevgen = temp
  for i in range( tn ):
    ngen[ i ] = union[ i, : 3 ]
  for i in range ( tn, N ):
    ngen [ i ] = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( 1, LOCI ) )
  gen = ngen

In [0]:


mutList = [ gimutation, mutation, gpmutation]
crossList = [ arithmeticCrossover, arithmeticCrossPerLoci, crossover ]
selList = [ selectionDetBT, selectionProbBT, selectionTrunc, selectionFibTrunc, selectionLR, selectionEliteLR ]
avg = np.empty((3,3,6))
#best = np.empty((3,3,6))
bestGene = np.empty((3,3,6), dtype=list)
stdv = np.empty((3,3,6))
def evol():
  global gen
  global avg
  global stdv
  global bestGene
  avgVector = np.empty(30)
  for m, mut in enumerate( mutList ):
    for c, cross in enumerate( crossList ):
      for s, sel in enumerate( selList ):
        for i in range( 30 ):
          gen = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( N, LOCI ) )
          tbstGene = list([5,5,5,75])
          bestGene[m][c][s] = tbstGene
          for _ in range( 50 ):
            sel()
            cross()
            mut()
          tbstGene = getBestIndividual(gen,N)
          avgVector[ i ] = np.mean(fitVector(gen, N))
        bestGene[m][c][s] = tbstGene if ( tbstGene[3] < bestGene[m][c][s][3] ) else bestGene[m][c][s]
        avg[m][c][s] = np.mean(avgVector)
        stdv[m][c][s] = np.std(avgVector)
      np.around( bestGene[m][c][s], 3, out=bestGene[m][c][s])


* **tn** -> number to keep during truncation selection
* **min, max** -> (0,2) linear rank parameters
* **en** -> number of top indivituals to keep for elite selection
* **Psbt** -> prob that the best indiviual is kept in binary tournament 
* **Pm** -> prob that loci is mutated
* **mr** -> range (**-mr**,** mr**) value of mutation for uniform mutaion
* **Pc**-> prob of crossover
* **alpha** -> % distributed for arithmetic crossover
* **shuffleflag** -> 1 = suffle before cross; 0 = no shuffle
* **pen** -> scaler value multiply by *fitness*
* **penFlag** -> enforce penalty 

In [0]:
tn = 8
min = .5
max = 2 - min
en = 2
Psbt = 0.8
Pm = 0.1
mr = 0.75
Pc = 0.8
alpha = 0.7
shuffFlag = 1
pen = 2
penFlag = 1

In [0]:
evol()


In [0]:
import pandas as pd
mutNames = [x.__name__ for x in mutList]
crossNames = [x.__name__ for x in crossList]
selNames = [x.__name__ for x in  selList]

index =pd.MultiIndex.from_product([mutNames, crossNames, selNames], names=['Mutation','Crossover','Selection'])#'Average','St.Dev.', 'Best'])

x1=[]
x2=[]
x3=[]
x4=[]
for i in range(3):
  for j in range(3):
    for k in range(6):
      x1.append(bestGene[i][j][k][0])
      x2.append(bestGene[i][j][k][1])
      x3.append(bestGene[i][j][k][2])
      x4.append( bestGene[i][j][k][3])

digits = 6
s = pd.Series(np.around(avg, digits).flatten(), index=index )
r = pd.Series(np.around(stdv, digits).flatten(), index=index )
p = pd.Series(np.around(x1, digits), index=index )
q = pd.Series(np.around(x2, digits),index=index )
w = pd.Series(np.around(x3, digits), index=index )
e = pd.Series(np.around(x4, digits), index=index )

data =pd.DataFrame()
data = pd.merge(s.to_frame(name='Average'), r.to_frame(name='St. Dev'), on=['Mutation', 'Selection','Crossover'])
data = pd.merge(data, p.to_frame(name='X1'), on=['Mutation', 'Selection','Crossover'])
data = pd.merge(data, q.to_frame(name='X2'), on=['Mutation', 'Selection','Crossover'])
data = pd.merge(data, w.to_frame(name='X3'), on=['Mutation', 'Selection','Crossover'])
data = pd.merge(data, e.to_frame(name='Fitness'), on=['Mutation', 'Selection','Crossover'])

X1, X2, X3, Fitness -> Best of Run


In [39]:
data

Average   St. Dev  \
Mutation   Selection         Crossover                                     
gimutation selectionDetBT    arithmeticCrossover      0.000024  0.000043   
           selectionProbBT   arithmeticCrossover      0.000302  0.000895   
           selectionTrunc    arithmeticCrossover      6.453325  3.219282   
           selectionFibTrunc arithmeticCrossover      3.998913  1.997264   
           selectionLR       arithmeticCrossover      0.007185  0.016609   
           selectionEliteLR  arithmeticCrossover      0.000016  0.000036   
           selectionDetBT    arithmeticCrossPerLoci   0.000000  0.000000   
           selectionProbBT   arithmeticCrossPerLoci   0.000000  0.000000   
           selectionTrunc    arithmeticCrossPerLoci   5.801529  1.647801   
           selectionFibTrunc arithmeticCrossPerLoci   3.208385  2.268686   
           selectionLR       arithmeticCrossPerLoci   0.000004  0.000006   
           selectionEliteLR  arithmeticCrossPerLoci   0.000000  0.000000   
           selectionDetBT    crossover                0.002978  0.004734   
           selectionProbBT   crossover                0.125038  0.534540   
           selectionTrunc    crossover                9.281592  2.843800   
           selectionFibTrunc crossover                4.751088  2.373332   
           selectionLR       crossover                0.067507  0.171739   
           selectionEliteLR  crossover                0.001893  0.005760   
mutation   selectionDetBT    arithmeticCrossover      0.075587  0.049566   
           selectionProbBT   arithmeticCrossover      0.095521  0.065784   
           selectionTrunc    arithmeticCrossover      5.838720  1.899399   
           selectionFibTrunc arithmeticCrossover      4.070546  1.769042   
           selectionLR       arithmeticCrossover      0.140913  0.112331   
           selectionEliteLR  arithmeticCrossover      0.095790  0.054087   
           selectionDetBT    arithmeticCrossPerLoci   0.071540  0.037705   
           selectionProbBT   arithmeticCrossPerLoci   0.089669  0.075707   
           selectionTrunc    arithmeticCrossPerLoci   5.664142  2.065166   
           selectionFibTrunc arithmeticCrossPerLoci   2.534217  1.452058   
           selectionLR       arithmeticCrossPerLoci   0.128508  0.061426   
           selectionEliteLR  arithmeticCrossPerLoci   0.085626  0.082185   
           selectionDetBT    crossover                0.122688  0.241161   
           selectionProbBT   crossover                0.131580  0.126085   
           selectionTrunc    crossover                8.350189  1.841327   
           selectionFibTrunc crossover                4.054368  1.746940   
           selectionLR       crossover                0.409178  0.498656   
           selectionEliteLR  crossover                0.103470  0.057472   
gpmutation selectionDetBT    arithmeticCrossover      0.000011  0.000028   
           selectionProbBT   arithmeticCrossover      0.000258  0.000594   
           selectionTrunc    arithmeticCrossover      8.133605  2.296756   
           selectionFibTrunc arithmeticCrossover      4.443397  2.622679   
           selectionLR       arithmeticCrossover      0.010791  0.026031   
           selectionEliteLR  arithmeticCrossover      0.000006  0.000012   
           selectionDetBT    arithmeticCrossPerLoci   0.000000  0.000000   
           selectionProbBT   arithmeticCrossPerLoci   0.000000  0.000000   
           selectionTrunc    arithmeticCrossPerLoci   6.251853  2.591732   
           selectionFibTrunc arithmeticCrossPerLoci   3.863060  2.445640   
           selectionLR       arithmeticCrossPerLoci   0.000015  0.000028   
           selectionEliteLR  arithmeticCrossPerLoci   0.000000  0.000000   
           selectionDetBT    crossover                0.002539  0.006410   
           selectionProbBT   crossover                0.014795  0.034504   
           selectionTrunc    crossover               11.184634  3.603552   
           selectionFibTrun